In [2]:
!nvidia-smi

Sun Nov 23 12:00:34 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   54C    P0              69W / 300W |   5274MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/workspace/Approach2


In [4]:
import torch
import numpy as np
from datasets import load_dataset
from evaluate import load as eval_load
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

In [ ]:
!huggingface-cli login --token {hf_token}

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `llm_finetuning` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llm_finetuning`


In [6]:
def postprocess_indictrans_text(preds, labels, source_lang, target_lang):
    prefix = f"{target_lang} {source_lang}"

    def clean(x):
        return x[len(prefix):].strip() if x.startswith(prefix) else x.strip()

    preds = [clean(p) for p in preds]
    labels = [[clean(l)] for l in labels]
    return preds, labels


def to_numpy(x):
    return x.cpu().numpy() if isinstance(x, torch.Tensor) else x


def compute_metrics_indictrans(eval_preds):
    preds = eval_preds.predictions
    labels = eval_preds.label_ids

    if isinstance(preds, tuple):
        preds = preds[0]

    preds = to_numpy(preds)
    labels = to_numpy(labels)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_indictrans_text(
        decoded_preds, decoded_labels, source_lang, target_lang
    )

    bleu = metric_bleu.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )["score"]

    chrf = metric_chrf.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )["score"]

    return {"bleu": bleu, "chrf": chrf}

In [14]:
model_checkpoint = "ai4bharat/indictrans2-en-indic-dist-200M"
source_lang = "eng_Latn"
target_lang = "mar_Deva"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, trust_remote_code=True)

In [8]:
dataset = load_dataset("thenlpresearcher/iitb_en_indic_marathi_orig_punct_tokenized")

train_ds = dataset["train"]
valid_ds = dataset["validation"]

In [9]:
metric_bleu = eval_load("sacrebleu")
metric_chrf = eval_load("chrf")

In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [15]:
# Take a small subset for quick testing
small_train = train_ds.select(range(200))
small_valid = valid_ds.select(range(50))

training_args = Seq2SeqTrainingArguments(
    output_dir="tmp-test-indictrans",
    eval_strategy="steps",
    save_strategy="steps",           # no saving, avoids clutter
    eval_steps=20,                # frequent eval to see metrics quickly
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,           # doesn't matter; dataset is tiny
    max_steps=40,                 # hard stop after ~40 steps
    predict_with_generate=True,
    logging_steps=10,             # see loss every few iterations
    load_best_model_at_end=True,
    push_to_hub=True,
    report_to="none",             # no WandB/TensorBoard
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics_indictrans,
)

trainer.train()

/tmp/ipykernel_74858/3118396278.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss,Bleu,Chrf
20,7.081900,7.051559,25.389304,52.350138
40,6.311800,6.706669,26.313253,53.234399


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=40, training_loss=7.04262638092041, metrics={'train_runtime': 16.0897, 'train_samples_per_second': 9.944, 'train_steps_per_second': 2.486, 'total_flos': 6940904914944.0, 'train_loss': 7.04262638092041, 'epoch': 0.8})

In [16]:
trainer.push_to_hub("thenlpresearcher/sample-en-mar-with-punct")

Processing Files (0 / 0): |                    |  0.00B /  0.00B            
Processing Files (3 / 4): 100%|███████████████▉|  851MB /  851MB,  300MB/s  
Processing Files (4 / 4): 100%|████████████████|  851MB /  851MB,  260MB/s  
Processing Files (4 / 4): 100%|████████████████|  851MB /  851MB,  229MB/s  
New Data Upload: 100%|█████████████████████████| 1.79MB / 1.79MB,  526kB/s  


CommitInfo(commit_url='https://huggingface.co/thenlpresearcher/tmp-test-indictrans/commit/23d84eff3795e86953246137f8929b74f9c9ae1e', commit_message='thenlpresearcher/sample-en-mar-with-punct', commit_description='', oid='23d84eff3795e86953246137f8929b74f9c9ae1e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/thenlpresearcher/tmp-test-indictrans', endpoint='https://huggingface.co', repo_type='model', repo_id='thenlpresearcher/tmp-test-indictrans'), pr_revision=None, pr_num=None)

In [ ]:
# training_args = Seq2SeqTrainingArguments(
#     output_dir="indictrans-marathi-finetuned",
#     eval_strategy="steps",
#     save_strategy="steps",
#     eval_steps=6000,
#     save_steps=6000,
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     predict_with_generate=True,
#     load_best_model_at_end=True,
#     save_total_limit=2,
#     push_to_hub=False,    # set True if needed
# )

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics_indictrans,
)

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub("your-username/your-model-name")